# コーディングをする前におこなったこと

①　Europa Year Book の写真を撮る。

②　Office　Lensで Word形式にする。

③　WordからWebノート形式に直す。

④　③で直したWebノートをExcelで開く。

⑤　先頭に行を追加し、Receiverの国名を入力する。行が国ごとに分かれていることを確認し、:のところは;にする。
　　

↑この⑤の作業は省略できないか検討中です

⑥　できたExcelをAzure NotebooksのLibrariesにアップロードする(CCODE_PYTHON.xlsx)。

⑦　ccode-kurizaki.xlsxを Librariesにアップロードする。

⑧　New Coding Instruction for Diplomatic Representation Dataに従い、

　　ランクをエクセルに記入してAzure NotebooksのLibrariesにアップロードする(SenderRank.xlsx)。



In [1]:
#コーディングで使うモジュールをインポートしていきます
# pandasのインポート
import pandas as pd
# numptyのインポート
import numpy as np
# 時間操作用ライブラリ
from time import sleep
#日本語を含むコメントがはじかれないように
#coding: utf-8

最初に、ccode-kurizaki.xlsxに関して処理していきます

In [2]:
#ccode-kurizaki.xlsxをinput_file_nameという名前のファイルにして操作しやすくします
input_file_name = 'ccode-kurizaki.xlsx'

#このファイルをinput_bookとして開きます
input_book = pd.ExcelFile(input_file_name) 

#sheet_namesメソッドでExcelブック内の各シートの名前をリストで取得します
input_sheet_name = input_book.sheet_names

#取得したリストをinput_sheet_dfという名前のデータフレームにして見やすくします
#input_sheet_name[0]は一枚目のシートを開くということです(二枚目のシートならinput_sheet_name[1]です)
#国番号(CCode)と国名(StateName)の列のみ取得するために、ccode-kurizaki.xlsxのB列からC列まで取得します
input_sheet_df = input_book.parse(input_sheet_name[0], 
                                  parse_cols 
                                  = "B:C",
                                  )
#できたinput_sheet_dfの中身を見てみます
input_sheet_df = input_sheet_df[['StateName','CCode']]
input_sheet_df

,StateName,CCode
0,United States of America,2.0
1,Canada,20.0
2,Bahamas,31.0
3,Cuba,40.0
4,Cuba,40.0
5,Haiti,41.0
6,Haiti,41.0
7,Dominican Republic,42.0
8,Dominican Republic,42.0
9,Jamaica,51.0


このままだと後で取得した外交文書にある国名と比較しにくいので、このinput_sheet_dfをSeriesの形式に直します

In [3]:
#まずzipを使い、input_sheet_dfのStateNameにある国名、CCodeにある国番号の順にきれいな辞書型セットを作ります
CountryCode_dic = dict([(i,a) for i, a in zip(input_sheet_df.StateName, input_sheet_df.CCode)])
#CountryCode_dicの中身を見てみます
CountryCode_dic
#この辞書型データをSeriesに直します
CountryCode = pd.Series(CountryCode_dic)
CountryCode

Afghanistan                 700.0
Albania                     339.0
Algeria                     615.0
Andorra                     232.0
Angola                      540.0
Antigua & Barbuda            58.0
Argentina                   160.0
Armenia                     371.0
Australia                   900.0
Austria                     305.0
Austria-Hungary             300.0
Azerbaijan                  373.0
Baden                       267.0
Bahamas                      31.0
Bahrain                     692.0
Bangladesh                  771.0
Barbados                     53.0
Bavaria                     245.0
Belarus                     370.0
Belgium                     211.0
Belize                       80.0
Benin                       434.0
Bhutan                      760.0
Bolivia                     145.0
Bosnia and Herzegovina      346.0
Botswana                    571.0
Brazil                      140.0
Brunei                      835.0
Bulgaria                    355.0
Burkina Faso  

ここでいったんccode-kurizaki.xlsxに関する準備は終わります

次に、SenderRank.xlsxに関して処理していきます

In [4]:
#以下ccode-kurizaki.xlsxのときと同じ手順です
#今回はファイルの名前をinput_file_name2にします
input_file_name2 = 'SenderRank.xlsx'
 
#input_book2として開きます
input_book2 = pd.ExcelFile(input_file_name2) 
 
#sheet_namesメソッドでExcelブック内の各シートの名前をinput_sheet_name2としてリストで取得します
input_sheet_name2 = input_book2.sheet_names

#dataフレームとしてinput_sheet_name2の一つ目のsheetを読み込みます（input_sheet_name2[0]）
input_sheet_df2 = input_book2.parse(input_sheet_name2[0])
#読み込んだシート名の確認します
print("Sheet name:", input_sheet_name2[0])
#input_sheet_df2の中身を見てみます
input_sheet_df2

Sheet name: Sheet1


,Rank,Code
0,Not assigned or appointed,0
1,Ambassador,1
2,Nuncio,1
3,Apostolic nuncio,1
4,Apostolic Nuncio,1
5,papal nuncio,1
6,Papal nuncio,1
7,Papal Nuncio,1
8,pro-nuncio,1
9,Pro nuncio,1


SenderRank.xlsxに関する準備は以上です

最後に、CCode_PYTHON.xlsxを取り込んで使いやすいように処理します

In [5]:
#以下ccode-kurizaki.xlsxのときと同じ手順です
#今回はファイルの名前をinput_file_name3にします
input_file_name3 = 'CCODE_PYTHON.xlsx'
 
#input_book3として開きます
input_book3 = pd.ExcelFile(input_file_name3) 
 
#sheet_namesメソッドでExcelブック内の各シートの名前をinput_sheet_name3としてリストで取得します
input_sheet_name3 = input_book3.sheet_names
 
#lenでinput_sheet_name3のシートの総数を確認します
num_sheet = len(input_sheet_name3)
 
#input_sheet_name3のシートの数とシートの名前のリストの表示
print ("Sheet の数:", num_sheet)
print (input_sheet_name3)

Sheet の数: 1
['Kazakhstan']


In [6]:
#dataフレームとしてinput_sheet_name3の一つ目のsheetを読み込みます（input_sheet_name3[0]）
input_sheet_df3 = input_book3.parse(input_sheet_name3[0])
#読み込んだシート名の確認します
print("Sheet name:", input_sheet_name3[0])
#できたデータフレームを確認します
input_sheet_df3 

Sheet name: Kazakhstan


,Kazakhstan,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,"Afghanistan; Almaty, Shalyapina 56, Hotel Molo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Armenia; 480075 Almaty, pr. Seifullina 579, 7t...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Australia; 480004 Almaty, Kazybek bi 20A; tel....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Bulgaria; Almaty, Makataeva 13A; tel. (3272) 3...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Canada; 480100 Almaty, Karasai Batyr 34; tel. ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"China, People's Republic; Almaty, Furmanova 13...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"Croatia; Almaty, Furmanova 110; tel. (3272) 62...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"Cuba; Almaty, Zenkova 70, kv. 23; tel. and fax...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"Czech Republic; Almaty, pr. Zhibek zholy 64; t...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"Egypt; 480100 Almaty, Zenkova 59; tel. (3272) ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#Unamed1, 2 ,,,という列はいらないので消去します
#parse_cols = 0は一列目だけとるという意味です(二列目だけをとるならparse_cols = 1)
input_sheet_df3 = input_book3.parse(input_sheet_name3[0], 
                                  skiprow = 0,
                                  parse_cols = 0
                                  )
input_sheet_df3

,Kazakhstan
0,"Afghanistan; Almaty, Shalyapina 56, Hotel Molo..."
1,"Armenia; 480075 Almaty, pr. Seifullina 579, 7t..."
2,"Australia; 480004 Almaty, Kazybek bi 20A; tel...."
3,"Bulgaria; Almaty, Makataeva 13A; tel. (3272) 3..."
4,"Canada; 480100 Almaty, Karasai Batyr 34; tel. ..."
5,"China, People's Republic; Almaty, Furmanova 13..."
6,"Croatia; Almaty, Furmanova 110; tel. (3272) 62..."
7,"Cuba; Almaty, Zenkova 70, kv. 23; tel. and fax..."
8,"Czech Republic; Almaty, pr. Zhibek zholy 64; t..."
9,"Egypt; 480100 Almaty, Zenkova 59; tel. (3272) ..."


In [8]:
#このinput_sheet_df3をコピーしたものをdfとします
df = input_sheet_df3.copy()
#dfの中身を見てみます(コピーなのでinput_sheet_df3と同じ内容です)
df

,Kazakhstan
0,"Afghanistan; Almaty, Shalyapina 56, Hotel Molo..."
1,"Armenia; 480075 Almaty, pr. Seifullina 579, 7t..."
2,"Australia; 480004 Almaty, Kazybek bi 20A; tel...."
3,"Bulgaria; Almaty, Makataeva 13A; tel. (3272) 3..."
4,"Canada; 480100 Almaty, Karasai Batyr 34; tel. ..."
5,"China, People's Republic; Almaty, Furmanova 13..."
6,"Croatia; Almaty, Furmanova 110; tel. (3272) 62..."
7,"Cuba; Almaty, Zenkova 70, kv. 23; tel. and fax..."
8,"Czech Republic; Almaty, pr. Zhibek zholy 64; t..."
9,"Egypt; 480100 Almaty, Zenkova 59; tel. (3272) ..."


現在KAZAKSTANという列に国名、都市、電話番号…という要素がすべて入ってしまっているのでこれを；ごとに分割します

In [9]:
#Kazakhstanという列の中で;がでてきたら次の列に振り分けるという動作をします
df = df["Kazakhstan"].str.split(';', expand=True)
#ここの列名'Kazakhstan'の名前を入力せずにできないか悩んでいます
#できた7列を適当にCountryname, Cityname, Telephonenumber, Faxnumber, Senderpost, Others, Remarksとします
df.columns = ['Countryname', 'Cityname', 'Telephonenumber', 'Faxnumber', 'SenderPost', 'Other', 'Remarks']
#できたデータフレームの中身を見てみます
df

,Countryname,Cityname,Telephonenumber,Faxnumber,SenderPost,Other,Remarks
0,Afghanistan,"Almaty, Shalyapina 56, Hotel Molodezhnaya",tel. (3272) 28-68-71,Chargé d'affaires a.i.,NAJIBULLAH ZIA RAKHMAN.,None,None
1,Armenia,"480075 Almaty, pr. Seifullina 579, 7th Floor",tel. and fax (3272) 69-29-08,Chargé d’affaires a.i.,ARMAN MELIKIAN.,None,None
2,Australia,"480004 Almaty, Kazybek bi 20A",tel. (3272) 63-94-18,Ambassador,DOUGLAS TOWNSEND.,None,None
3,Bulgaria,"Almaty, Makataeva 13A",tel. (3272) 30-27-55,Chargé d'affaires a.i.,YORDAN TRENCHEV.,None,None
4,Canada,"480100 Almaty, Karasai Batyr 34",tel. (3272) 50-11-51,Ambassador,CHARLES RICHARD MANN.,None,None
5,"China, People's Republic","Almaty, Furmanova 137",tel. (3272) 63-49-66,fax (3272) 63-92-91,Ambassador,CHEN Dl.,None
6,Croatia,"Almaty, Furmanova 110",tel. (3272) 62-57-03,telex 251435,fax (3272) 50-62-92,Chargé d'affaires a.i.,KARINO HROMIN.
7,Cuba,"Almaty, Zenkova 70, kv. 23",tel. and fax (3272) 61-59-25,Chargé d'affaires a.i.,ROBERTO CRUZ GALINDO.,None,None
8,Czech Republic,"Almaty, pr. Zhibek zholy 64",tel. (3272) 33-47-13,fax (3272) 33-50-88,Chargé d'affaires a.i.,ALEXANDER LANGER.,None
9,Egypt,"480100 Almaty, Zenkova 59",tel. (3272) 60-16-22,fax (3272) 61-10-22,Ambassador,AYMAN HAMDI EL KOUNI.,None


In [10]:
#このdfをコピーしたものをdf2とします(コピーなのでdfと同じ内容です)
df2 = df.copy()
#df2の要素の中でtel,fax,telex,(3272)、数字が入っている要素を削除します
ret = df2.apply(lambda d: d.str.contains('tel|fax|(3272)|telex|\d', na=False, regex=True))
df2 = df2.mask(ret)
#Citynameの項目を消します
df2 = df2.drop("Cityname", axis=1)
#一列まるまるNaNと入っている物も消します
df2 = df2.dropna(axis = 1, how = 'all')
#df2の中身を見てみます
df2

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


,Countryname,Faxnumber,SenderPost,Other,Remarks
0,Afghanistan,Chargé d'affaires a.i.,NAJIBULLAH ZIA RAKHMAN.,None,None
1,Armenia,Chargé d’affaires a.i.,ARMAN MELIKIAN.,None,None
2,Australia,Ambassador,DOUGLAS TOWNSEND.,None,None
3,Bulgaria,Chargé d'affaires a.i.,YORDAN TRENCHEV.,None,None
4,Canada,Ambassador,CHARLES RICHARD MANN.,None,None
5,"China, People's Republic",NaN,Ambassador,CHEN Dl.,None
6,Croatia,NaN,NaN,Chargé d'affaires a.i.,KARINO HROMIN.
7,Cuba,Chargé d'affaires a.i.,ROBERTO CRUZ GALINDO.,None,None
8,Czech Republic,NaN,Chargé d'affaires a.i.,ALEXANDER LANGER.,None
9,Egypt,NaN,Ambassador,AYMAN HAMDI EL KOUNI.,None


次に、このデータフレームのCountrynameにある国名だけを抽出しSeriesにしていきます

In [11]:
#まずdf2のCountrnameという列にある国名をCountrylistというきれいなリストにします
Countrylist = df2["Countryname"].values.tolist()
#この複製をとり、Series形式に直します
Countrylist2 = Countrylist.copy()
Country = pd.Series(Countrylist2)
#Countryの中身を見てみます
Country

0                             Afghanistan
1                                 Armenia
2                               Australia
3                                Bulgaria
4                                  Canada
5                China, People's Republic
6                                 Croatia
7                                    Cuba
8                          Czech Republic
9                                   Egypt
10                                 France
11                                Georgia
12                                Germany
13                               Holy See
14                                Hungary
15                                  India
16                                   Iran
17                                 Israel
18                                  Italy
19                                  Japan
20    Korea, Democratic People's Republic
21                        Korea, Republic
22                             Kyrgyzstan
23                                

ここで、国名と国番号をまとめたデータフレームをdf3とします

In [12]:
#CountryCodeとCountryを比べdf3に格納します
df3 = CountryCode[CountryCode.index.isin(Country)]
#整理し、列名をSenderCountry, SenderCodeとします
df3 = df3.reset_index().rename(columns={'index':'SenderCountry'})
df3.columns = ['SenderCountry', 'SenderCode']
#df3の中身を見てみます
df3

,SenderCountry,SenderCode
0,Afghanistan,700.0
1,Armenia,371.0
2,Australia,900.0
3,Bulgaria,355.0
4,Canada,20.0
5,Croatia,344.0
6,Cuba,40.0
7,Czech Republic,316.0
8,Egypt,651.0
9,France,220.0


ここで、df2でCountrynameに入っていたのに、df3のSenderCountryにおいてはじかれてしまった国を探します

In [13]:
#比較をわかりやすくするため、df2のCountrnameだけをとったデータフレームdf4を作ります
df4 = pd.DataFrame(df2["Countryname"])
#df4のCountrynameとdf3のSenderCountryとを比べ、異なるもののみ残します
df5 = df4[~df4["Countryname"].isin(df3['SenderCountry'])]
df5

,Countryname
5,"China, People's Republic"
20,"Korea, Democratic People's Republic"
21,"Korea, Republic"
34,Ukraine
36,USA


調べてみると、Ukraine以外はccode-kurizaki.xlsxに少し異なる国名で登録されていたことがわかりました

Ukraineは、スキャンした時に(スペース)Ukraineとして認識されていました

そこで、ccode-kurizaki.xlsxにおいてChina, People's Republic、Korea, Democratic People's Republic、Korea, Republic、USAも新たな国名として追加し(ccode-kurizaki-REVISED.xlsx)、CCODE_PYTHON.xlsxではUkraineの前に入っているスペースをなくします(CCODE_PYTHON-REVISED.xlsx)

これを新たに特定のLibrariesにアップロードします

In [14]:
#特定のLibrariesにアップロード後、修正したエクセルファイル二つをまた開いて整理しSeries形式にします
#まずはccode-kurizaki-REVISED.xlsxのデータに関して、全ての変数に_newを付け加えて作り直します
input_file_name_new = 'ccode-kurizaki-REVISED.xlsx'
input_book_new = pd.ExcelFile(input_file_name_new) 
input_sheet_name_new = input_book_new.sheet_names
input_sheet_df_new = input_book_new.parse(input_sheet_name_new[0], 
                                  parse_cols 
                                  = "B:C",
                                  )
input_sheet_df_new = input_sheet_df_new[['StateName','CCode']]
CountryCode_dic_new = dict([(i,a) for i, a in zip(input_sheet_df_new.StateName, input_sheet_df_new.CCode)])
CountryCode_new = pd.Series(CountryCode_dic_new)

In [15]:
#次にCCODE-PYTHON-REVISED.xlsxのデータに関して、全ての変数に_newに付け加えて作り直します
input_file_name3_new = 'CCODE_PYTHON-REVISED.xlsx'
input_book3_new = pd.ExcelFile(input_file_name3_new) 
input_sheet_name3_new = input_book3_new.sheet_names
num_sheet_new = len(input_sheet_name3_new)
print ("Sheet の数:", num_sheet_new)
print (input_sheet_name3_new)
input_sheet_df3_new = input_book3_new.parse(input_sheet_name3_new[0])
print("Sheet name:", input_sheet_name3_new[0])
input_sheet_df3_new = input_book3_new.parse(input_sheet_name3_new[0], 
                                  skiprow = 0,
                                  parse_cols = 0
                                  )
df_new = input_sheet_df3_new.copy()
df_new = df_new["Kazakhstan"].str.split(';', expand=True)
df_new.columns = ['Countryname', 'Cityname', 'Telephonenumber', 'Others1', 'Others2', 'Others3', 'Others4']
df2_new = df_new.copy()
ret_new = df2_new.apply(lambda d: d.str.contains('tel|fax|(3272)|telex|\d', na=False, regex=True))
df2_new = df2_new.mask(ret_new)
#Citynameの列は使わないので消します
df2_new = df2_new.drop("Cityname", axis = 1)
#全ての行にNaNが入っているものは消します
df2_new = df2_new.dropna(axis = 1, how = 'all')
# NaNやNomeが入っているところは0を入れ、後々の処理でErrorが出ないよう処理しておきます
df2_new = df2_new.fillna(0)
#df_newの中身を見てみます
df2_new

Sheet の数: 1
['Kazakhstan']
Sheet name: Kazakhstan


/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:18: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


,Countryname,Others1,Others2,Others3,Others4
0,Afghanistan,Chargé d'affaires a.i.,NAJIBULLAH ZIA RAKHMAN.,0,0
1,Armenia,Chargé d’affaires a.i.,ARMAN MELIKIAN.,0,0
2,Australia,Ambassador,DOUGLAS TOWNSEND.,0,0
3,Bulgaria,Chargé d'affaires a.i.,YORDAN TRENCHEV.,0,0
4,Canada,Ambassador,CHARLES RICHARD MANN.,0,0
5,"China, People's Republic",0,Ambassador,CHEN Dl.,0
6,Croatia,0,0,Chargé d'affaires a.i.,KARINO HROMIN.
7,Cuba,Chargé d'affaires a.i.,ROBERTO CRUZ GALINDO.,0,0
8,Czech Republic,0,Chargé d'affaires a.i.,ALEXANDER LANGER.,0
9,Egypt,0,Ambassador,AYMAN HAMDI EL KOUNI.,0


In [16]:
Countrylist_new = df2_new["Countryname"].values.tolist()
Country_new = pd.Series(Countrylist_new)
#CountryCodeとCountryを比べdf3に格納します
df3_new = CountryCode_new[CountryCode_new.index.isin(Country_new)]
#整理し、列名をSenderCountry, SenderCodeとします
df3_new = df3_new.reset_index().rename(columns={'index':'SenderCountry'})
df3_new.columns = ['SenderCountry', 'SenderCode']
df3_new

,SenderCountry,SenderCode
0,Afghanistan,700.0
1,Armenia,371.0
2,Australia,900.0
3,Bulgaria,355.0
4,Canada,20.0
5,"China, People's Republic",710.0
6,Croatia,344.0
7,Cuba,40.0
8,Czech Republic,316.0
9,Egypt,651.0


読み取った全ての国に国番号が示されました

続いて、国番号と同じようにSenderRankのデータフレームも作ります

In [17]:
# df4 より Rank名->数値の変換用のSeriesデータを作成
conv = input_sheet_df2.set_index('Rank').Code
# 行毎にループ
for idx, row in df2_new.iterrows():
    # Others1以降の列を変換データにて変換
    tmp = row['Others1':].map(conv)
    # 上の処理で変換できなかったデータは削除
    tmp = tmp.dropna()
    if tmp.size > 0:
        df2_new.loc[idx, 'SenderRank'] = tmp[0]

In [18]:
# 不要なOthersの列を削除します
df2_new.drop(['Others1','Others2','Others3','Others4'], axis=1)


,Countryname,SenderRank
0,Afghanistan,3.0
1,Armenia,3.0
2,Australia,1.0
3,Bulgaria,3.0
4,Canada,1.0
5,"China, People's Republic",1.0
6,Croatia,3.0
7,Cuba,3.0
8,Czech Republic,3.0
9,Egypt,1.0


これも、最初NaNと表示されてしまう国がいくつかありました。調べてみると、.がスキャンした時に読み取れていなかったり

Rank名の前に空白が入っているなどのエラーでした

ですので、.が入っていないランク名もSenderRank.xlsxに追加し、空白を削除して実行をやり直しました

それでは、先程のdf3_newに他の列を付け加えて最終的にdf5としてまとめます

In [19]:
#df_newですでに分かれていた都市名Citynameの列ををくっつけます
df5 = pd.concat([df3_new, df_new['Cityname']], axis=1)
#ReceiverCountryには、国名を記入しますが、Excelのシート名でもあるので以前定義したように記します
df5['ReceiverCountry']= input_sheet_name3_new[0]
#ReceiverCodeには、ReceiverCountryの国番号を自動的に出すようにします
df5['ReceiverCode'] = CountryCode_dic_new[input_sheet_name3_new[0]]
#私が今回用いたEuropa Year Bookの年・巻番号を記しました
df5['Volume_Year'] = "1997Ⅱ"
df5 = pd.concat([df5, df2_new['SenderRank']], axis = 1)
df5

,SenderCountry,SenderCode,Cityname,ReceiverCountry,ReceiverCode,Volume_Year,SenderRank
0,Afghanistan,700.0,"Almaty, Shalyapina 56, Hotel Molodezhnaya",Kazakhstan,705.0,1997Ⅱ,3.0
1,Armenia,371.0,"480075 Almaty, pr. Seifullina 579, 7th Floor",Kazakhstan,705.0,1997Ⅱ,3.0
2,Australia,900.0,"480004 Almaty, Kazybek bi 20A",Kazakhstan,705.0,1997Ⅱ,1.0
3,Bulgaria,355.0,"Almaty, Makataeva 13A",Kazakhstan,705.0,1997Ⅱ,3.0
4,Canada,20.0,"480100 Almaty, Karasai Batyr 34",Kazakhstan,705.0,1997Ⅱ,1.0
5,"China, People's Republic",710.0,"Almaty, Furmanova 137",Kazakhstan,705.0,1997Ⅱ,1.0
6,Croatia,344.0,"Almaty, Furmanova 110",Kazakhstan,705.0,1997Ⅱ,3.0
7,Cuba,40.0,"Almaty, Zenkova 70, kv. 23",Kazakhstan,705.0,1997Ⅱ,3.0
8,Czech Republic,316.0,"Almaty, pr. Zhibek zholy 64",Kazakhstan,705.0,1997Ⅱ,3.0
9,Egypt,651.0,"480100 Almaty, Zenkova 59",Kazakhstan,705.0,1997Ⅱ,1.0


最後に、作成したデータフレームdf5をCSVに保存します

In [20]:
#pandas形式の場合.to_csvというメソッドでCSV化できます
#Excelで開きたい場合、encoding="SHIFT-JIS"という引数を追加してSHIFT-JISでエンコーディングするのが最適です
df5.to_csv("CCode.csv", index=False, encoding="'Shift_JISx0213'")

Libraryの中に入っていればcsvとして保存できています(Libraryのページを開きなおしてください)

国名と国番号、外交官ランクのコーディングは以上です

以下にこれから取り組もうと思っていることを記します

-----------------------------------------------------------------------------------------------------------------

ここまで見てきた一つ目のシートへの処理を全てのシートに対して行い、さらに各シートのデータフレームを一つのデータフレームに結合します。 全てのシートに対してforでループを回せばよいので、以下のようにすればできます。
#データフレームの取扱、Excelファイルのデータフレームへの読み込み用
import pandas as pd
#正規表現
import re
#input file name
input_file = 'CCODE-PYTHON-REVISED.xls'
#xlsx book Open
input_book = pd.ExcelFile(input_file) 
#xlsファイルのシートの名前をリストとして取得
input_sheet_name = input_book.sheet_names
#シートの総数
num_sheet = len(input_sheet_name)
output_df = pd.DataFrame(index=[], columns=[])
for i_sheet in range(0, num_sheet) :
    #シートを格納しているリストから読み込み
    input_sheet_df = input_book.parse(input_sheet_name[i_sheet],
                                      skiprows = 0,
                                      parse_cols = O
                                      )
#ccode-kurizaki-REVISED.xlsxなどはどの国にも共通なので通常通りに開く    
    
    for i_m in range(1, len(input_sheet_df.columns)) : 
    #データフレームの列について 1列目からループを回す     
        
        ...あとは一緒
   
処理して全てのシートについて結合したデータフレームを表示します。 

-----------------------------------------------------------------------------------------------------------------

SA,VAなどのところは太田さんと相談して進めようかと思っています



# 改善の余地があるところ

冒頭の⑤の作業だけでなく、そのときに空白を削除しなくてもよい方法を考え中です

データフレームから辞書に直すときに、国名をわざわざ入力しなくてもよい方法も模索しています

そのほかの点でもまだ気づいていないところがあるかと思いますので、ぜひアドバイスをお願いします！